# 🧠 Clasificación y Limpieza de Datos con Verificación de Ruta

## 📂 Verificar y cargar el archivo desde ruta relativa

In [1]:
import pandas as pd
import os

ruta = './datos/dataset_ejemplo_1300.csv'

if not os.path.exists(ruta):
    print(f"❌ El archivo no existe en la ruta: {ruta}")
else:
    df_raw = pd.read_csv(ruta)
    print("✅ Archivo cargado correctamente")
    display(df_raw.head())

❌ El archivo no existe en la ruta: ./datos/dataset_ejemplo_1300.csv


## 🎯 Crear variable binaria `Alta_conectividad`

In [2]:
df_raw['Alta_conectividad'] = (df_raw['Horas_Internet'] > 3.5).astype(int)

NameError: name 'df_raw' is not defined

## 🧩 Crear variable categórica `Nivel_conectividad`

In [ ]:
def clasificar_conectividad(horas):
    if horas <= 1:
        return 0
    elif horas <= 3.5:
        return 1
    else:
        return 2

df_raw['Nivel_conectividad'] = df_raw['Horas_Internet'].apply(clasificar_conectividad)

## 🧼 Imputación de valores faltantes con `SimpleImputer`

Cuando una columna contiene valores faltantes (`NaN`), se pueden aplicar diferentes estrategias para imputar los datos:

| Estrategia         | Qué hace                                              | Cuándo usarla                                      | Ejemplo práctico                                                                   |
|--------------------|--------------------------------------------------------|----------------------------------------------------|--------------------------------------------------------------------------------------|
| `'mean'`           | Sustituye `NaN` con la **media**                      | Datos numéricos sin outliers                       | Edad: `[25, 28, NaN, 22, 30] → NaN → `26.25`                                        |
| `'median'`         | Sustituye `NaN` con la **mediana**                   | Datos numéricos con valores extremos              | Ingreso: `[1000, 1200, NaN, 8000, 1100] → NaN → `1150`                             |
| `'most_frequent'`  | Sustituye `NaN` con el **valor más frecuente**        | Datos categóricos o discretos                     | Género: `['M', 'F', NaN, 'F'] → NaN → `'F'`                                          |
| `'constant'`       | Sustituye `NaN` con un **valor fijo** definido        | Para identificar o normalizar nulos explícitamente| Ciudad: `['Bogotá', NaN, 'Cali'] con fill_value='Desconocida' → NaN → `'Desconocida'` |

In [ ]:
from sklearn.impute import SimpleImputer

df_clean = df_raw.copy()

# Imputación
imputer_edad = SimpleImputer(strategy='median')
df_clean['Edad'] = imputer_edad.fit_transform(df_clean[['Edad']])

imputer_ingreso = SimpleImputer(strategy='mean')
df_clean['Ingreso'] = imputer_ingreso.fit_transform(df_clean[['Ingreso']])

## 🔣 Codificación de variables categóricas

In [ ]:
df_clean = pd.get_dummies(df_clean, drop_first=True)

## 📏 Escalado seguro de variables numéricas

In [ ]:
from sklearn.preprocessing import MinMaxScaler

cols_numericas = df_clean.select_dtypes(include=['int64', 'float64']).columns
scaler = MinMaxScaler()
df_clean[cols_numericas] = scaler.fit_transform(df_clean[cols_numericas])

## 🤖 Entrenamiento del modelo

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X = df_clean.drop(columns=['Alta_conectividad'])
y = df_clean['Alta_conectividad']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("✅ Exactitud del modelo:", round(accuracy, 4))